<a href="https://colab.research.google.com/github/barisuke/barisuke.github.io/blob/master/emg%E5%89%8D%E5%87%A6%E7%90%8620200821%E6%B1%BA%E5%AE%9A%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!pip install neurokit2
!pip install japanize-matplotlib

     |████████████████████████████████| 993kB 7.2MB/s 
     |████████████████████████████████| 4.1MB 9.5MB/s 
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-cp36-none-any.whl size=4120275 sha256=ccda473dfb86cc2cf07e604a23852b1c5495db5206dd13b3f858397541dfa1ba
  Stored in directory: /root/.cache/pip/wheels/b7/d9/a2/f907d50b32a2d2008ce5d691d30fb6569c2c93eefcfde55202
Successfully built japanize-matplotlib


In [ ]:
import os
os.chdir('/content/drive/My Drive/sensor_analysis/notebooks')
print(os.getcwd())

import sys
sys.path.append('/content/drive/My Drive/sensor_analysis/src')
import hs_detect as hsd

from glob import glob
from os.path import join, exists
import json
from collections import OrderedDict
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
import pickle
from scipy import signal 
import neurokit2 as nk

%load_ext autoreload
%autoreload 2

FileNotFoundError: ignored

In [ ]:
X_COLUMNS = [11, 12, 13, 14, 15, 16, 23, 24, 25]
Y_COLUMNS = [26]

In [ ]:
def adjacent_diffs(X: np.array) -> np.array:
  '''隣り合う2項の差'''
  return X[1:]-X[:-1] 
 
def moving_rms(data, n):
    data_sq = data ** 2
    window = np.ones(n) / n
    return np.sqrt(np.convolve(data_sq, window, mode='same'))

def process_emg(path, mvc100=False, fs=1500, interval=5, winlen=50):
    """
    Args:
        fs (int): sampling frequency [Hz].
        mvc100(float): 100%mvc of before right(left)01 [μV].
        interval (int): duration of trimmed EMG data.
        winlen (int): duration of window for smoothing [ms].
    """
    try:
        df = pd.read_csv(path, skiprows=3)
    except:
        print('UserWarning: no data in {}'.format(path))
        return np.zeros(fs*interval), np.zeros(fs*interval)
    df.drop(df.columns[[1, 2, -1]], axis=1, inplace=True)
    N, D = df.shape
    if D == 3:  # 前脛骨筋のみ、2チャネル
        col_idx = 1 if 'right' in path else 2
    else:  # 4チャネル
        if 'vm' in path or 'rf' in path or 'daitai' in path:
            col_idx = 1 if 'right' in path else 2
        else:
            col_idx = 3 if 'right' in path else 4

    # trim `interval` sec (by default 5sec x 1,500 Hz = 7,500 samples)
    # 生筋電波形の取得
    raw = df.iloc[:, col_idx].values
    # 配列上の5秒間の長さを計算
    interval_sample = interval * fs
    # 配列上の50msecの移動RMS窓の長さを計算
    subsample = winlen * fs // 1000
    # 移動RMSによる平滑化
    smoothed = moving_rms(raw, subsample)
    minidx = signal.argrelmin(smoothed, order=100)[0] #極小値 tupleなので0を指定
    diff_max_idx = minidx[np.argmax(adjacent_diffs(smoothed[minidx]))] # 直後の項との差が最大となるインデックス  
    """
    x = np.array(range(smoothed.shape[0]))
    print("minidx",minidx )
    print("x[diff_max_idx]", x[diff_max_idx])
    print("diff_max_idx", diff_max_idx)
    print("adjacent_diffs(smoothed[minidx])", adjacent_diffs(smoothed[minidx]))
    plt.plot(x,smoothed,'k-',label='元系列')
    plt.plot(x[minidx],smoothed[minidx],'bo',label='ピーク値（最小）')
    plt.plot(x[diff_max_idx],smoothed[diff_max_idx],'ro',label='start')
    plt.plot(x[diff_max_idx+interval_sample],smoothed[diff_max_idx+interval_sample],'yo',label='end')
    plt.legend()
    plt.show()
    """
    cut_smoothed = smoothed[diff_max_idx:diff_max_idx+interval_sample]
    if mvc100:
        # 100%mvcが引数にある時(before right01以外全てこっち)
        # 正規化
        normalized = cut_smoothed / mvc100 * 100 
        # 5秒間の総和を返す
        return np.sum(normalized[:interval_sample])
    else:
        # 100%mvcが引数にない時
        # 正規化に使用する100%mvcを取得
        num=int(1*fs) #移動平均の個数
        b=np.ones(num)/num
        mva=np.convolve(cut_smoothed, b, mode='same')#移動平均
        mvc100 = np.max(mva)
        print(mvc100)
        # 正規化
        normalized = cut_smoothed / mvc100 * 100
        # 5秒間の総和と100%mvcを返す
        return np.sum(normalized[:interval_sample]), mvc100  

def calc_mf(freqaxis, PS):
    # 中間周波数を計算
    import bisect
    cumsum = np.cumsum(PS)
    half_idx = bisect.bisect_left(cumsum, cumsum[-1] / 2)
    return freqaxis[half_idx]


# Prepare dataset

In [ ]:
true_labels = {
    "20200708" : {
        "before" : 5642589, "after01" : 551150, "after02" : 431654, "after03" : 457053
    },
    "20200709" : {
        "before" : 638314, "after01" : 596670, "after02" : 553434, "after03" : 444519 
    },
    "20200713" : {
        "before" : 663155, "after01" : 512743, "after02" : 457900, "after03" : 436924, "after04" : 508547
    },
    "20200717" : {
        "before" : 624272, "after01" : 580883, "after02" : 580168, "after03" : 521613, "after04" : 663759
    },
    "20200720" : {
        "before" : 631516, "after01" : 545252, "after02" : 579524, "after03" : 611121, "after04" :644903
    },
    "20200724" : {
        "before" : 688702, "after01" : 498042, "after02" : 444639, "after03" : 463018, "after04" : 411785
    },
    "20200727" : {
        "before" : 635678, "after01" : 425771, "after02" : 485359, "after03" : 407195, "after04" : 500406
    },

    "20200817" : {
        "before" : 626920 , "after01" :   565769 , "after02" :  650666 
    },
    "20200819" : {
        "before" :  671503 , "after01" :  589653 , "after02" :  544994 
    },
}
#true_labels = json.dumps(true_labels)
true_labels

{'20200708': {'after01': 551150,
  'after02': 431654,
  'after03': 457053,
  'before': 5642589},
 '20200709': {'after01': 596670,
  'after02': 553434,
  'after03': 444519,
  'before': 638314},
 '20200713': {'after01': 512743,
  'after02': 457900,
  'after03': 436924,
  'after04': 508547,
  'before': 663155},
 '20200717': {'after01': 580883,
  'after02': 580168,
  'after03': 521613,
  'after04': 663759,
  'before': 624272},
 '20200720': {'after01': 545252,
  'after02': 579524,
  'after03': 611121,
  'after04': 644903,
  'before': 631516},
 '20200724': {'after01': 498042,
  'after02': 444639,
  'after03': 463018,
  'after04': 411785,
  'before': 688702},
 '20200727': {'after01': 425771,
  'after02': 485359,
  'after03': 407195,
  'after04': 500406,
  'before': 635678},
 '20200817': {'after01': 565769, 'after02': 650666, 'before': 626920},
 '20200819': {'after01': 589653, 'after02': 544994, 'before': 671503}}

### 各日の100%mvcを計算
emg_paths = sorted(glob(join('/content/drive/My Drive/202006～_筋電計＋センサデータ取得@KERNEL/*/03_*/matsui/*before*_right01.csv')))
date_mvc100 = {}
for path in emg_paths:
  date = path.split("/")[-4]
  try:
    val, mvc100 = process_emg(path)
  except Exception as e:
    print(date, e)
    continue
  date_mvc100[date] = mvc100
date_mvc100

# Create dataset

In [ ]:
#csv_paths = sorted(glob(join('/content/drive/My Drive/202006～_筋電計＋センサデータ取得@KERNEL/*/02_*/tomita/*.csv')))
#csv_paths.extend(sorted(glob(join('/content/drive/My Drive/202006～_筋電計＋センサデータ取得@KERNEL/*/02_*/matsui/*.csv'))))
csv_paths = sorted(glob(join('/content/drive/My Drive/202006～_筋電計＋センサデータ取得@KERNEL/20200709/02_*/matsui/*.csv'))) + sorted(glob(join('/content/drive/My Drive/202006～_筋電計＋センサデータ取得@KERNEL/202008*/02_*/matsui/*.csv')))  
name = "matsui"
X, Y = [], []
yname_list = []
date_mvc100 = {}
#result_dic = {}
for csv_path in csv_paths:
    key = csv_path.split('/')[-1].split('.')[0]
    if key == 'before01':
        key = 'before'
    person = csv_path.split('/')[-2]
    date = csv_path.split("/")[-4]
    
    # fetch output data (EMG data)
    root_dir = '/'.join(csv_path.split('/')[:-3])
    emg_paths = glob(join(root_dir, f'03_*/{name}/*{key}_right01.csv'))
    before_right01_paths = glob(join(root_dir, f'03_*/{name}/*before*_right01.csv'))
    for path in before_right01_paths:
        ps_sum_temp, mvc100 = process_emg(path)
        date_mvc100[date] = mvc100
    ps_sums = []
    for path in emg_paths:
        try:
          ps_sum_temp = process_emg(path, mvc100=date_mvc100[date])
          ps_sums.append(ps_sum_temp)
          print(path, ps_sum_temp)
        except Exception as e:
          raise e
    try:
      ps_sums = np.array(ps_sums)
      ps_sum = np.mean(ps_sums[~np.isnan(ps_sums)])
    except Exception as e:
      print(e, csv_path)
      continue
    """

    # 西出さん作成の教師ラベルを読み込む
    ps_sum = 0
    if date in true_labels.keys():
      if key in true_labels[date].keys():
        ps_sum = true_labels[date][key]
    if ps_sum == 0:
      continue
    """
    """
    # load input data (9-axis sensor data)
    print('processing', csv_path)
    df = pd.read_csv(csv_path)
    df['ps_sum'] = ps_sum
    x, y = hsd.create_resampled_gaitcycle(df, X_COLUMNS, Y_COLUMNS, lr='right', n_points=100)
    X.append(x)
    Y.append(y)

    # name list of y
    yname_list.append(f"{person}_{date}_{key}")
    """


500.5832307605401
/content/drive/My Drive/202006～_筋電計＋センサデータ取得@KERNEL/20200709/03_パワースペクトラム/matsui/2020-07-09-12-18_20200709_after01_right01.csv 586030.2382883311
500.5832307605401
/content/drive/My Drive/202006～_筋電計＋センサデータ取得@KERNEL/20200709/03_パワースペクトラム/matsui/2020-07-09-20-30_20200709_after02_right01.csv 550677.6495966661
500.5832307605401
/content/drive/My Drive/202006～_筋電計＋センサデータ取得@KERNEL/20200709/03_パワースペクトラム/matsui/2020-07-09-21-53_20200709_after03_right01.csv 442844.3546251993
500.5832307605401
/content/drive/My Drive/202006～_筋電計＋センサデータ取得@KERNEL/20200709/03_パワースペクトラム/matsui/2020-07-09-10-19_20200709_before_right01.csv 641400.2070717392


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


%matplotlib inline
import matplotlib.pyplot as plt
plt.plot(list(result_dic.keys()), list(result_dic.values()))
plt.xticks(rotation=90)
plt.rcParams['figure.figsize'] = (100 ,100)
plt.show()